<a href="https://colab.research.google.com/github/lescai-teaching/class-lt-biology/blob/master/classes/19_python-alignments/19_biopython_allineamenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alignments with Biopython

## Install the necessary tools

First of all we need to install biopython: remember we use _!_ to tell the virtual machine that we are executing a terminal command and **not** a python command

In [2]:
! pip install biopython

     |████████████████████████████████| 2.3 MB 6.7 MB/s 


Then we need to install some additional software:

- we begin by installing a package manager



In [3]:
! pip install -q condacolab
import condacolab
condacolab.install_miniconda()

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py37_4.9.2-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:24
🔁 Restarting kernel...


- then we use this package manager to install a multiple alignment software

In [1]:
! conda install -c bioconda -y mafft=7.310 muscle=3.8.1551

Solving environment: / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - mafft
    - muscle


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.4.26  |       h06a4308_0         124 KB
    certifi-2021.10.8          |   py37h06a4308_2         151 KB
    conda-4.12.0               |   py37h06a4308_0        14.5 MB
    mafft-7.310                |       he1b5a44_3         2.9 MB  bioconda
    muscle-3.8.1551            |       hc9558a2_5         280 KB  bioconda
    openssl-1.1.1n             |       h7f8727e_0         2.5 MB
    ------------------------------------------------------------
                                           Total:        20.4 MB

The following NEW packages will be INSTALLED:

  mafft              bioconda/linux-64::mafft-7.310-he1b5a44_3
  muscle             bioconda/linux-64::muscle-3.8.155

We verify the software is installed correctly, by typing the command of one of the software we need.

In [5]:
! muscle


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.


Basic usage

    muscle -in <inputfile> -out <outputfile>

Common options (for a complete list please see the User Guide):

    -in <inputfile>    Input file in FASTA format (default stdin)
    -out <outputfile>  Output alignment in FASTA format (default stdout)
    -diags             Find diagonals (faster for similar sequences)
    -maxiters <n>      Maximum number of iterations (integer, default 16)
    -maxhours <h>      Maximum time to iterate in hours (default no limit)
    -html              Write output in HTML format (default FASTA)
    -msf               Write output in GCG MSF format (default FASTA)
    -clw               Write output in CLUSTALW format (default FASTA)
    -clwstrict         As -clw, with 'CLUSTAL W (1.81)' header
    -log[a] <logfile>  Log to file (append if -loga, overwrite if -log)
 

## Global and Local Alignment

First of all, we use the _Seq_ Class in Python to manage sequence formats together with the additional metadata they might need.

Then, we create two sequences objects which we can use to explore the functionality of biopython in terms of aligning and formatting sequences.

In [24]:
from Bio.Seq import Seq
seqA = Seq("PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK")
seqB = Seq("PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFGNPVIPFKDGIYFAATEKSNVRGWVFGSTMNNKSQ")

The Biopython package also contains libraries to perform pairwise alignments: we can import both this library as well as a sub-class used to format sequences.

We do this, to be able to call the functions directly without the nested structure of the class.

In [25]:
import Bio
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

### Local Alignment

Now that we have imported all the classes we need, we can first run a local alignment.

Alignment functions all have the format _.align.localXY_ and _.align.globalXY_
where XY are two letters added to the name of the function to indicate the alignment scoring method to be used.

In particular:

- the first letter (X in our example above) indicates the **parameters for matches**:
  - x = no parameters
  - m = parameters are passed explicitly as arguments in the function call (one for matches score, one for mismatches score)
  - d = a dictionary is passed in the call for any possible match
  - c = a function is created to pass the score

- the second letter (Y in our example above) indicates the **parameters for gap** and it follows a similar logic:
  - x = no parameters
  - s = parameters are passed explicitly as arguments in the function call (one for gap opening score, one for gap extension score): in this case scores are identical for both sequences
  - d = similar to the above _s_ BUT scores are different for the two sequences, i.e. we need to pass 4 arguments
  - c = a function is created to pass the gap penalties

When choosing _d_ in the matches section we can pass a scoring matrix, like one of the _blosum_ for example.

See the manual page [here](https://biopython.org/docs/1.75/api/Bio.pairwise2.html)

In the example below we choose to score both matches and gaps, and use the same scores for both sequences (_m_ is the first letter and _s_ is the second letter).

In [26]:
local_alignment = pairwise2.align.localms(seqA, seqB, 2, -1, -10, -1)

Now the result can be a list of many possible alignments.

In order to print the results, we need a _for loop_, and in this case we also want to limit the number of alignments printed so we introduce an index variable and limit the number of loops with that one.

In [27]:
index = 1
for a in local_alignment:
  if index < 5:
    print(format_alignment(*a, full_sequences=True))
    index += 1

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|     |.|||.||.||.|||.|||||                              
PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHT-----FGNPVIPFKDGIYFAATEKSNVRGWVFGSTMNNKSQ---------------
  Score=46

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|.|     |||.||.||.|||.|||||                              
PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFG-----NPVIPFKDGIYFAATEKSNVRGWVFGSTMNNKSQ---------------
  Score=46



### Global Alignment

Next, we can use the same scores as above, and perform a global alignment instead, using the _.align.globalXY_ family of functions.

The function uses the last 2 letters in the very same way as the local alignment function.

We perform the alignment as below:

In [28]:
global_alignment = pairwise2.align.globalms(seqA, seqB, 2, -1, -10, -1)

And in the very same way, we inspect the alignment using a _for loop_ and limit the number of alignments printed to four:

In [29]:
index = 1
for a in global_alignment:
  if index < 5:
    print(format_alignment(*a, full_sequences=True))
    index += 1

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|     |.|||.||.||.|||.||||| .|||.||.|...|.|              
PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHT-----FGNPVIPFKDGIYFAATEKSN-VRGWVFGSTMNNKSQ--------------
  Score=22

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|.|     |||.||.||.|||.||||| .|||.||.|...|.|              
PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHTFG-----NPVIPFKDGIYFAATEKSN-VRGWVFGSTMNNKSQ--------------
  Score=22

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|     |.|||.||.||.|||.|||||. |||.||.|...|.|              
PDEIFRSDTLYLTQDLFLPFYSNVTGFHTINHT-----FGNPVIPFKDGIYFAATEKSNV-RGWVFGSTMNNKSQ--------------
  Score=22

PDKVFRSSVLHSTQDLFLPFFSNVTWFHAISGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIVNNATNVVIK
||..|||..|..||||||||.||||.||.|..|.|     |||.||.||.|||.|||||. |||

By comparing the two alignments above, we can discuss the differences between global and local alignments.


# Multiple Sequence Alignment


## Download the data

In order to work with MSA, we will use one of the multiple sequences FASTA of the exercises, which we can download to the virtual machine using _wget_

In [8]:
! wget https://raw.githubusercontent.com/lescai-teaching/class-lt-biology/master/exercises/strain-classification/strain01/esercizio_01.fasta

--2022-05-05 16:45:14--  https://raw.githubusercontent.com/lescai-teaching/class-lt-biology/master/exercises/strain-classification/strain01/esercizio_01.fasta
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46751 (46K) [text/plain]
Saving to: ‘esercizio_01.fasta’

esercizio_01.fasta  100%[===================>]  45.66K  --.-KB/s    in 0.007s  

2022-05-05 16:45:15 (6.84 MB/s) - ‘esercizio_01.fasta’ saved [46751/46751]



## Import the Classes

Biopython does not have a class to perform MSA, like it does for pairwise alignments. 

But we have several classes which allow to format the command line neede to execute an external software (one of those we installed at the beginning of this class).

First we import the class:

In [6]:
from Bio.Align.Applications import MuscleCommandline

Then, we can use the function to format the command line, indicating both inputs and outputs, as well as additional parameters

In [ ]:
muscle_cline = MuscleCommandline(input="esercizio_01.fasta", 
                                 out="aligned.fasta", 
                                 diags = True, 
                                 maxiters = 1, 
                                 log="align_log.txt")

We can inspect what the function creates, in terms of command executed on the computer by using a _print_



In [10]:
print(muscle_cline)

muscle -in esercizio_01.fasta -out aligned.fasta -diags -log align_log.txt -maxiters 1


Now we can execute the command, and run the multiple sequence alignment:

In [9]:
muscle_cline()

('',
 '\nMUSCLE v3.8.1551 by Robert C. Edgar\n\nhttp://www.drive5.com/muscle\nThis software is donated to the public domain.\nPlease cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.\n\nesercizio_01 11 seqs, lengths min 4212, max 4212, avg 4212\n00:00:00     38 MB(9%)  Iter   1    1.52%  K-mer dist pass 1\n00:00:00     38 MB(9%)  Iter   1  100.00%  K-mer dist pass 1\n00:00:00     38 MB(9%)  Iter   1    1.52%  K-mer dist pass 2\n00:00:00     38 MB(9%)  Iter   1  100.00%  K-mer dist pass 2\n00:00:00    42 MB(10%)  Iter   1   10.00%  Align node       \n00:00:00    77 MB(19%)  Iter   1   20.00%  Align node\n00:00:01    82 MB(20%)  Iter   1   30.00%  Align node\n00:00:01    85 MB(21%)  Iter   1   40.00%  Align node\n00:00:01    86 MB(21%)  Iter   1   50.00%  Align node\n00:00:01    88 MB(21%)  Iter   1   60.00%  Align node\n00:00:02    89 MB(21%)  Iter   1   70.00%  Align node\n00:00:02    94 MB(23%)  Iter   1   80.00%  Align node\n00:00:02    95 MB(23%)  Iter   1   90.00%  Align node\n00

The results will be stored in the file indicated as output in the command above.

We will now use the class _AlignIO_ in order to read alignment files.

In [12]:
from Bio import AlignIO
alignment = AlignIO.read("aligned.fasta", "fasta")

Alignments formats are also a list of multiple objects, which we can print again with a _for_ loop as below:

In [13]:
for record in alignment:
  print("%s - %s" % (record.id, record.seq))

beta_hCoV-19/England/CAMB-1AE30A/2020 - ------------------------------------------------------ATGTCATGCATGCAAATTACATATTTTGGAGGAATACAAATCCAATTCAGTTGTCTTCCTATTCTTTATTTGACATGAGTAAATTTCCCCTTAAATTAAGGGGTACTGCTGTTATGTCTTTAAAAGAAGGTCAAATCAATGATATGATTTTATCTCTTCTTAGTAAAGGTAGACTTATAATTAGAGAAAACAACAGAGTTGTTATTTCTAGTGATGTTCTTGTTAACAACTAAACGAACAATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTTACAACCAGAACTCAATTACCCCCTGCATACACTAATTCTTTCACACGTGGTGTTTATTACCCTGACAAAGTTTTCAGATCCTCAGTTTTACATTCAACTCAGGACTTGTTCTTACCTTTCTTTTCCAATGTTACTTGGTTCCATGCTATACATGTCTCTGGGACCAATGGTACTAAGAGGTTTGCTAACCCTGTCCTACCATTTAATGATGGTGTTTATTTTGCTTCCACTGAGAAGTCTAACATAATAAGAGGCTGGATTTTTGGTACTACTTTAGATTCGAAGACCCAGTCCCTACTTATTGTTAATAACGCTACTAATGTTGTTATTAAAGTCTGTGAATTTCAATTTTGTAATGATCCATTTTTGGGTGTTTATTACCACAAAAACAACAAAAGTTGGATGGAAAGTGAGTTCAGAGTTTATTCTAGTGCGAATAATTGCACTTTTGAATATGTCTCTCAGCCTTTTCTTATGGACCTTGAAGGAAAACAGGGTAATTTCAAAAATCTTAGGGAATTTGTGTTTAAGAATATTGATGGTTATTTTAAAATATATTCTAAGCACACGCCTATTAATTTAGTGCGTGATCTCCCTCAGGGTTTTTCGGCT

The output above shows the alignment in a nicely formatted way, but in the order chosen during the progressive alignment step.

Next, we want to create a phylogenetic tree from the MSA.

In order to do this, we need to import:

- a phylogenetic class
- a class to help us calculate the distances from the MSA
- a class that provides the functions to calculate and draw the tree

In [14]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor

With this in mind, we can now decide which Matrix to use (Blosum90 in our example), then calculate get the distances from the MSA and construct the tree using the _UPGMA_ method.

Finally, we can plot the tree:

In [22]:
calculator = DistanceCalculator('blosum90')
dm = calculator.get_distance(alignment)
constructor = DistanceTreeConstructor()
tree = constructor.upgma(dm)

Phylo.draw_ascii(tree)

  _________________________________ gamma_hCoV-19/Brazil/AM-L70-CD1723/2020
 |
 |                                 , gamma_hCoV-19/Japan/IC-0563/2021
 |                                 |
 |                                 , delta_hCoV-19/England/CAMC-14A5112/2021
 |                              ___|
_|                             |   | delta_hCoV-19/USA/NY-PRL-2021_03_29_0...
 |                             |   |
 |                             |   |, epsilon_hCoV-19/USA/CA-CZB-13243/2020
 |                             |   ,|
 |                             |   || epsilon_hCoV-19/USA/CA-CZB-13247/2020
 |                             |   |
 |_____________________________|   | beta_hCoV-19/Sweden/20-51765/2020
                               |   |
                               |   |, alpha_hCoV-19/England/MILK-A7C3B1/2020
                               |   ||
                               |   || alpha_hCoV-19/England/CAMC-A64871/2020
                               |    |
                    